In [1]:
%pip install pyhwpx


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\thdco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [34]:
from pyhwpx import Hwp
import win32com.client as win32
import pandas as pd
hwp = Hwp()
hwp.Open(r"C:\Users\thdco\OneDrive\Documents\GitHub\pyhwpx\dongtemplate.hwp") # 주간행사계획 한글파일 상대경로

True

In [35]:
def time_column_initail(): 
    hwp.set_pos(4,0,0)
    hwp.TableCellBlock() #그냥 테이블 셀 선택
    hwp.TableCellBlockExtend() #선택된셀에서 f5누른거랑 같은 기능
    hwp.TableColPageDown()
    hwp.set_cur_field_name("time")
time_column_initail()
#시간 column 전부다 선택하는 기능

In [6]:
#데이터 전처리
df = pd.read_excel(r"C:\Users\thdco\OneDrive\Documents\GitHub\pyhwpx\dongdummy.xlsx")
#엑셀에서 더미데이터 read
df = df.replace(r'\n', '', regex=True)
# 데이터안에 \n 다지우기

df = df.fillna(" ")  # NaN 값을 으로 채우기
df.tail()

,date,time,dong,eventName,place,personnel,phone
27,11.8.(금),14:00:00,상대동,지역사회보장협의체 공방수업,행정복지센터,25.0,황지연010-0020-2000
28,11.8.(금),16:00:00,온천2동,지역사회보장협의체『11월의 산타』추진,관내 저소득가구,23.0,박은심010-0000-0000
29,11.9.(토),08:30:00,노은2동,송림마을4단지 경로당 가을야유회,경북삼천포(4단지 경로당 출발),25.0,김홍식010-0010-0000
30,11.9.(토),14:50:00,진 잠 동,통장협의회 하반기 워크숍,중국 장가계(행정복지센터 출발),30.0,공광열010-0001-0101
31,11.10.(일),,,,,,


In [7]:
date_counts = df['date'].value_counts()
date_counts

date
11.6.(수)     8
11.5.(화)     7
11.8.(금)     7
11.7.(목)     6
11.9.(토)     2
11.4.(월)     1
11.10.(일)    1
Name: count, dtype: int64

In [27]:
import re
#위에처럼 count순으로 sort된다 
#string 날짜데이터를 날짜순으로 sort하는 코드
date_counts = df['date'].value_counts().reset_index()
date_counts.columns = ['date', 'count']  # 열 이름 지정

# 월과 일만 추출하는 함수 정의
def extract_month_day(date_str):
    match = re.search(r'(\d+)\.\s*(\d+)', date_str)
    if match:
        return int(match.group(1)), int(match.group(2))  # 월, 일 반환
    return None

# 'month', 'day' 열 추가
date_counts[['month', 'day']] = date_counts['date'].apply(lambda x: pd.Series(extract_month_day(x)))

# 월과 일 기준으로 정렬
date_counts = date_counts.sort_values(by=['month', 'day']).drop(columns=['month', 'day']).reset_index(drop=True)

#date 컬럼만
date_column_df = date_counts[['date']]

# 정렬된 결과 출력
print(date_counts)

        date  count
0   11.4.(월)      1
1   11.5.(화)      7
2   11.6.(수)      8
3   11.7.(목)      6
4   11.8.(금)      7
5   11.9.(토)      2
6  11.10.(일)      1


In [28]:
# print(count_value)


for index, row in date_counts.iterrows():
    pset = hwp.HParameterSet.HTableDeleteLine
    hwp.move_to_field(f'time{{{{{0+2*index}}}}}')
    print(index)
    count = row['count']
    
    if count>2 :
        for i in range(count - 2):
            hwp.TableAppendRow()
            print(f"{index}" + ": append" )
    elif (count == 2):
        continue
    else :
        hwp.TableLowerCell()
        hwp.HAction.GetDefault("TableDeleteRow", pset.HSet)
        #TLqkf
        hwp.HAction.Execute("TableDeleteRow", pset.HSet)
        print(f"{index}"+": delete" )
        
        
    

0
0: delete
1
1: append
1: append
1: append
1: append
1: append
2
2: append
2: append
2: append
2: append
2: append
2: append
3
3: append
3: append
3: append
3: append
4
4: append
4: append
4: append
4: append
4: append
5
6
6: delete


In [32]:
#date 제외 데이터 넣기
df_no_date = df.drop(columns=['date'])
hwp.set_pos(4,0,0)
# 셀 전부 선택하는 코드 ->
hwp.TableCellBlock()
hwp.TableCellBlockExtend()
hwp.TableColEnd()
hwp.TableColPageDown()
# 선택된 셀을 imsi필드로 바꾸고 
hwp.set_cur_field_name("imsi")

hwp.put_field_text("imsi", df_no_date.values.flatten().tolist())


In [ ]:
# date 데이터 넣기
for index, row in date_column_df.iterrows():
    hwp.move_to_field(f'date{{{{{index}}}}}')
    hwp.insert_text(row['date'])
    print(row['date'])